In [244]:
from random import randint, random
import numpy as np
from collections import defaultdict
from statistics import mean
from tqdm.notebook import tqdm
from collections import Counter

<img src="algorithm.png">

In [266]:
class black_Jack:
    def __init__(self):
        '''
            deck - бесконечная колода карт
            p - стратегия для всех состояний. 0 - "еще", 1 - "хватит"
                Состояние зависит от трех факторов (наличие играющего туза (0, 1); 
                                                    открытая карта сдающего (1-10); 
                                                    количество очков (4-21))
            Q - оценка функции ценности. Зависит от состония и действия
            returns - сохраняет доходы для каждой пары состояние-действие после каждого эпизода
        '''
        self.deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1] 
        self.Q =  defaultdict(lambda: [0.0, 0.0])
        self.p = defaultdict(lambda: 0)
        self.returns = defaultdict(lambda: (list(), list()))
    
    def start(self, dealer_card):
        # принимает на вход значение карты сдающего и возвращает начальное значение состояния
        
        card1 = self.deck[randint(0,len(deck)-1)]
        card2 = self.deck[randint(0,len(deck)-1)]
        
        # оценка туза
        T = 0
        if 1 in [card1, card2]:
            T = self.playing_T(sum([card1, card2])-1)
            
        # подсчет очков
        points = sum([card1, card2]) + T*10
        
        return (T, dealer_card, points)
    
    def playing_T(self, points):
        if points + 11 <= 21:
            return 1
        return 0
    
    def play(self, dealer_card):
        S_A = []
        s = self.start(dealer_card)
        points = s[2]
        T = s[0]
        
        while True:
            a = self.p[s]
            
            # action 0 - взять еще карту
            if a == 0:
                another_card = self.deck[randint(0,len(deck)-1)]
                
                # оценка туза
                if T == 0 and another_card == 1:
                    T = self.playing_T(points)
                    another_card += T*10
                
                #подсчет очков
                points += another_card
                
                # если перебор, то переоцениваем туза
                if points > 21 and T == 1:
                    points -= 10
                    T = 0
            
            S_A.append((s, a))
            if a == 1:
                return S_A, 0, points
                
            r = self.reward(points)
            
            if r != 0:
                return S_A, r, points
            
            s = (T, dealer_card, points)
    
    def reward(self, points, p_dealer=0):
        
        # награда при action 0 (взять еще карту) - оценка количества очков
        if p_dealer == 0:
            if  points < 21:
                return 0
            if points == 21:
                return 1
            return -1
        
        # награда при action 1 (хватит) - сравнение количества очков с очками диллера
        if points == p_dealer:
            return 0
        if points > p_dealer or p_dealer > 21:
            return 1
        return -1
        
    def policy_improvement(self, S_A, r, points=0, p_dealer=0, e=0.1):
        
        # подсчет последнего reward, если последний action был 1
        if p_dealer:
            r = self.reward(points, p_dealer)
        
        
        S_A_set = set(S_A) # для оценки уникальных пар состояние-действие
        for s, a in S_A_set:
            self.returns[s][a].append(r) # доход равен награде, так как обесценивание = 1, а доход зависит только от конечной награды
            self.Q[s][a] = mean(self.returns[s][a]) # средний доход для пары состояние-действие
            A = np.argmax(self.Q[s]) # отбираем лучший action 
            if random() < e: # обновляем стратегию на лучший action с вероятностью 1-е
                self.p[s] = 1-A
            self.p[s] = A
        
        return self.p, r

In [267]:
deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 1] 
agent = black_Jack()
R = []
e = 0.5

In [268]:
for n in tqdm(range(50000)):
    if n//1000 == 0:
        e /= 2
    e = 0.3
    dealer_card = deck[randint(0,len(deck)-1)]
    S_A, r, points = agent.play(dealer_card)
    if r == 0:
        p_dealer = dealer_card
        while True:
            p_dealer += deck[randint(0,len(deck)-1)]
            if p_dealer > 17:
                break
        p, r = agent.policy_improvement(S_A, r, points, p_dealer, e=e)
    p, r = agent.policy_improvement(S_A, r, e=e)
    
    R.append(r)

In [269]:
Counter(R)

Counter({-1: 25951, 1: 21218, 0: 2831})

In [202]:
x, y, z = tuple(zip(*p))

In [207]:
actions = p.values()